Уважаемый студент!


…Итак, данные собраны и можно начинать анализ. Коллеги посоветовали вам выделить паттерны и на их основе построить торговую стратегию.


Вы решили посмотреть сначала самые простые паттерны на основе скользящих средних и затем дополнить их более сложными шаблонами, такими как паттерны разворота, продолжения и свечного анализа.


Также вы понимаете, что нужен инструмент оценки эффективности и поскольку, у вас уже есть данные прошлых периодов, вы решаете создать инструмент оценки эффективности новой стратегии, пока у вас есть немного свободного времени.

На основании вышесказанного, вам необходимо построить модель, которая будет принимать решение о торговых операциях по одной или группе бумаг и криптовалют, и оценить ее эффективность на отложенных данных.


Вам предлагается на основе представленной информации:


Создать код на Python, который разделит ваши данные на тренировочный, тестовый и валидационный наборы данных.
Построить одну или несколько моделей на основе паттернов технического анализа, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам.
Провести подбор гиперпараметров моделей с использованием обучающей и тестовой выборок.
Провести финальное тестирование построенных торговых стратегий на валидационном наборе данных и сравнить их между собой.
Сформировать дашборд, показывающий эффективность различных стратегий во времени.

In [1]:
%pip install backtesting
%pip install nbformat
from assets.DataProvider import DataProvider
from assets.HW2Strategy import HW2Strategy_SMA_RSI, HW2Strategy_SMA
from assets.HW2Backtest import HW2Backtest
from assets.enums import DataResolution, DataPeriod
from backtesting import Strategy
import nbformat
import datetime as dt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


c:\Users\Owner\anaconda3\envs\otus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
g:\Work\Repos\otus_homework\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Initialize data provider with BTC/USDT data
data_provider = DataProvider(
    tickers=['BTC/USDT'],
    resolution=DataResolution.DAY_01,
    period=DataPeriod.YEAR_05
)

# Load the data
if not data_provider.data_load():
    data_provider.data_request()
    data_provider.data_save()

btc_data = data_provider.data['BTC/USDT']
backtester: HW2Backtest = HW2Backtest(btc_data)


In [3]:
param_grid = {
    'sma_short': [5, 8, 10, 12, 15],
    'sma_long': [15, 20, 25, 30],
    'rsi_period': [10, 14, 21],
    'rsi_upper': [65, 70, 75, 80],
    'rsi_lower': [20, 25, 30, 35]
}

param_grid = {
    'sma_short': [5, 15],
    'sma_long': [15, 30],
    'rsi_period': [10, 21],
    'rsi_upper': [65, 80],
    'rsi_lower': [20, 35]
}

In [4]:
def test_strategy(strategy_name: str, strategy_class: Strategy, param_grid, backtester: HW2Backtest):
    print(f'Running {strategy_name} strategy validation...')
    best_params = backtester.get_best_strategy(strategy_class, param_grid)
    validation_stats = backtester.backtest_strategy(
        backtester.validation_data,
        strategy_class,
        best_params[0]
    )
    return validation_stats
    


In [5]:
# Strategy SMA
strategy_name = 'SMA'
strategies_results_sma = test_strategy(strategy_name, HW2Strategy_SMA, param_grid, backtester)
strategies_results = {f'{strategy_name}': strategies_results_sma}


Running SMA strategy validation...
Testing 32 parameter combinations...


32it [00:01, 22.82it/s]



Best parameters found:
4.20: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 10, 'rsi_upper': 65, 'rsi_lower': 20}
4.20: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 10, 'rsi_upper': 65, 'rsi_lower': 35}
4.20: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 10, 'rsi_upper': 80, 'rsi_lower': 20}
4.20: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 10, 'rsi_upper': 80, 'rsi_lower': 35}
4.20: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 21, 'rsi_upper': 65, 'rsi_lower': 20}

Best score: 4.20

Best strategy performance:
Return: 0.14%
Trades count: 95
Sharpe Ratio: 0.18
Max Drawdown: -0.45%
Win Rate: 40.00%


In [6]:
# Strategy SMA_RSI
strategy_name = 'SMA_RSI'
strategies_results_sma_rsi = test_strategy(strategy_name, HW2Strategy_SMA_RSI, param_grid, backtester)
strategies_results[strategy_name] = strategies_results_sma_rsi


Running SMA_RSI strategy validation...
Testing 32 parameter combinations...


32it [00:01, 24.41it/s]



Best parameters found:
6.56: {'sma_short': 15, 'sma_long': 30, 'rsi_period': 21, 'rsi_upper': 80, 'rsi_lower': 35}
6.52: {'sma_short': 15, 'sma_long': 30, 'rsi_period': 21, 'rsi_upper': 65, 'rsi_lower': 35}
6.51: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 10, 'rsi_upper': 65, 'rsi_lower': 20}
5.81: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 21, 'rsi_upper': 65, 'rsi_lower': 20}
5.55: {'sma_short': 5, 'sma_long': 15, 'rsi_period': 10, 'rsi_upper': 65, 'rsi_lower': 35}

Best score: 6.56

Best strategy performance:
Return: 0.07%
Trades count: 11
Sharpe Ratio: 0.56
Max Drawdown: -0.02%
Win Rate: 63.64%


In [7]:
# Create performance dashboard
backtester.create_performance_dashboard(strategies_results)
